In [1]:
!pip install timm

     |████████████████████████████████| 244 kB 3.0 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import os
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
from torch.autograd import Variable

import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score

from collections import OrderedDict
import timm

In [3]:
# store the path to the directories with preprocessed png images
train_dir = '../input/siic-isic-224x224-images/train/'
test_dir = '../input/siic-isic-224x224-images/test/'

# load csv files with image name and metadata
train_df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test_df = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')

### Create Dataset and DataLoader

In [4]:
# set device to gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the Dataset and the DataLoader
class MyDataset(Dataset):
    def __init__(self, dataframe, train=True, transform=None):
        self.df = dataframe
        self.train = train
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df['image_name'][idx]
        
        if self.train:
            img_path = train_dir + img_name + '.png'
        else:
            img_path = test_dir + img_name + '.png'
        
        # read in the image
        image = cv2.imread(img_path) # (224, 224, 3)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # reorder colors
        image = transforms.ToPILImage()(image) 
            
        if self.transform:
            image = self.transform(image)
          
        if self.train:
            label = self.df['target'][idx]
            return image, label
        else:
            return image

# TODO: add additional data augmentation
data_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

trainset = MyDataset(train_df, train=True, transform=data_transform)
testset = MyDataset(test_df, train=False, transform=test_transform)

### Define a multilayer neural net

In [5]:
num_inputs = 150528 # 3 x 224 x 224 color images
num_outputs = 2

class MultiLayerNet(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_units):
        super(MultiLayerNet, self).__init__()
        self.linear1 = nn.Linear(num_inputs, hidden_units)
        self.linear2 = nn.Linear(hidden_units, num_outputs)

    def forward(self, input):
        input = input.view(-1, num_inputs)
        output = self.linear1(input)
        output = torch.tanh(output)
        output = self.linear2(output)
        return output
    
##########################################################

class ConvolutionalNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(ConvolutionalNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5) # 3 channel, 16 feature maps, 5x5 square convolution
        self.conv2 = nn.Conv2d(16, 128, 5) # 16 input, 128 output, 5x5 square convolution
        
        self.linear1 = nn.Linear(128 * 53 * 53, 64)  # 64 hidden units
        self.linear2 = nn.Linear(64, num_outputs)  # 64 hidden units to 10 output units

    def forward(self, input):
        output = F.tanh(self.conv1(input))
        output = F.max_pool2d(output, (2, 2))   # 2 by 2 max pooling (subsampling) 
        output = F.tanh(self.conv2(output))
        output = F.max_pool2d(output, (2, 2))   # 2 by 2 max pooling (subsampling) 
        
        # flatten to vector
        output = output.view(-1, self.num_flat_features(output)) # flatten features
        output = self.linear1(output)
        output = F.tanh(output)
        output = self.linear2(output)
        return output

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### Try a CNN architecture based off of LeNet

In [6]:
# Create a CNN based off of LeNet


class C1(nn.Module):
    def __init__(self):
        super(C1, self).__init__()

        self.c1 = nn.Sequential(OrderedDict([
            ('c1', nn.Conv2d(3, 6, kernel_size=(5, 5))),
            ('relu1', nn.ReLU()),
            ('s1', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))

    def forward(self, img):
        output = self.c1(img)
        return output


class C2(nn.Module):
    def __init__(self):
        super(C2, self).__init__()

        self.c2 = nn.Sequential(OrderedDict([
            ('c2', nn.Conv2d(6, 16, kernel_size=(5, 5))),
            ('relu2', nn.ReLU()),
            ('s2', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))

    def forward(self, img):
        output = self.c2(img)
        return output


class C3(nn.Module):
    def __init__(self):
        super(C3, self).__init__()

        self.c3 = nn.Sequential(OrderedDict([
            ('c3', nn.Conv2d(16, 120, kernel_size=(5, 5))),
            ('relu3', nn.ReLU())
        ]))

    def forward(self, img):
        output = self.c3(img)
        return output


class F4(nn.Module):
    def __init__(self):
        super(F4, self).__init__()

        self.f4 = nn.Sequential(OrderedDict([
            ('f4', nn.Linear(120*49*49, 84)),
            ('relu4', nn.ReLU())
        ]))

    def forward(self, img):
        output = self.f4(img)
        return output


class F5(nn.Module):
    def __init__(self):
        super(F5, self).__init__()

        self.f5 = nn.Sequential(OrderedDict([
            ('f5', nn.Linear(84, 2))  # 
        ]))

    def forward(self, img):
        output = self.f5(img)
        return output


class LeNet(nn.Module):
    """
    Input - 3x224x224
    Output - 2
    """
    def __init__(self):
        super(LeNet, self).__init__()

        self.c1 = C1()
        self.c2_1 = C2() 
        self.c2_2 = C2() 
        self.c3 = C3() 
        self.f4 = F4() 
        self.f5 = F5() 

    def forward(self, img):
        output = self.c1(img)

        x = self.c2_1(output)
        output = self.c2_2(output)

        output += x

        output = self.c3(output)
        # flatten to a vector
        output = output.view(-1, self.num_flat_features(output)) # flatten features
        output = self.f4(output)
        output = self.f5(output)
        return output
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### Use EfficientNet for pretraining and then finetune

https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

- Initialize the pretrained model
- Reshape the final layer(s) to have the same number of outputs as the number of classes in the new dataset
- Define for the optimization algorithm which parameters we want to update during training
- Run the training step

In [7]:
# # Use pretrained Resnet
# network = models.resnet18(pretrained=True)
# network.fc = nn.Linear(512, 2)   # replace the last fully connected layer

# # check if CUDA is available
# train_on_gpu = torch.cuda.is_available()

# if train_on_gpu:
#     network.cuda()

### Train the neural network model

In [25]:
def train(network, epoch, train_loader, verbose=True):
    network.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if verbose and batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())   # add the loss of each batch
    return losses

def test(network, train_loader):
    network.eval()
    test_loss = 0
    correct = 0
    probs = np.zeros((len(train_loader.dataset)))
    targets = np.zeros((len(train_loader.dataset)))
    
    for i, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        output = network(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        prob = F.softmax(output, 1)[:, 1].cpu().detach().numpy()
        probs[i*batch_size:(i+1)*batch_size] = prob
        targets[i*batch_size:(i+1)*batch_size] = target.cpu().detach().numpy()

    test_loss /= len(train_loader.dataset)
    
    roc = roc_auc_score(targets, probs)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%), AUROC: {:.4f}\n'.format(
        test_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset), roc))
    return test_loss, 100. * correct / len(train_loader.dataset), roc

In [9]:
# %%time

# batch_size = 32
# img_size = (224, 224)
# epochs = 10       # number of epochs to train
# lr = 0.001        # learning rate

# # load data
# train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
# test_loader  = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

# # reset model
# # network = MultiLayerNet(num_inputs, num_outputs, hidden_units=1000)
# # network = ConvolutionalNet(num_inputs, num_outputs)
# # network = LeNet()

# # Use a pretrained EfficientNet
# network = timm.create_model('tf_efficientnet_b2_ns', 
#                           pretrained=True, 
#                           num_classes=2)


# # check if CUDA is available
# train_on_gpu = torch.cuda.is_available()
# if train_on_gpu:
#     network.cuda()

# # TODO: also try AdamW optimizer and dynamic learning rate
# optimizer = optim.SGD(network.parameters(), lr=lr)

# train_losses = []
# for epoch in range(epochs):
#     train_loss = train(network, epoch, train_loader)
#     train_losses += train_loss

# # display the decreasing training loss
# plt.plot(train_losses)
# # test_loss, test_acc = test(test_loader)  # print the validation error
# # plt.hlines(test_loss, 0, len(train_losses), color='r', label='Test Loss')
# plt.legend(loc='upper right')
# plt.title('Training Loss over Epochs')

### Trying Stratified KFold

In [ ]:
%%time

batch_size = 32
img_size = (224, 224)
epochs = 10       # number of epochs to train
lr = 0.001        # learning rate

skf = StratifiedKFold(5, shuffle=True, random_state=0)
fold = 0
rocs = []

for train_index, test_index in skf.split(train_df['image_name'], train_df['target']):
    if fold == 0:
        train_single = train_index
        test_single = test_index
    
    fold += 1
    print("Fold: ", fold)
    PATH = "enet{}.pt".format(fold)
    
    # node that the testset here is actually the validation set
    train_rows = train_df.loc[train_index]
    test_rows = train_df.loc[test_index]
    train_rows.reset_index(drop=True, inplace=True)
    test_rows.reset_index(drop=True, inplace=True)
    
    # set up data
    trainset = MyDataset(train_rows, train=True, transform=data_transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    # note that the testset here does include labels because it's the validation set
    testset = MyDataset(test_rows, train=True, transform=test_transform)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=4)
   
    # set up model
    network = timm.create_model('tf_efficientnet_b2_ns', pretrained=True, num_classes=2)
    train_on_gpu = torch.cuda.is_available()
    if train_on_gpu:
        network.cuda()
        
    optimizer = optim.SGD(network.parameters(), lr=lr)
    best_roc = 0
    counter = 0
    
    # train model
    for epoch in range(epochs):
        train_loss = train(network, epoch, train_loader)
        # print("Train loss: ", train_loss)
        
        # TODO: try early stopping
        test_loss, test_acc, test_roc = test(network, test_loader)
        print("Eval loss: ", test_loss)
        print("Eval AUROC: ", test_roc)
        if test_roc >= best_roc:
            best_roc = test_roc
            torch.save(network.state_dict(), PATH)  # save the state of the best model so far
        else:
            counter += 1
            if counter > 1:
                print('No improvement in ROC for two consecutive epochs')
                break
    print("Best AUROC: ", best_roc)
    rocs.append(best_roc)
    
    

Fold:  1
Train Epoch: 0 [0/26500 (0%)]	Loss: 2.200136
Train Epoch: 0 [3200/26500 (12%)]	Loss: 0.376369
Train Epoch: 0 [6400/26500 (24%)]	Loss: 0.100226
Train Epoch: 0 [9600/26500 (36%)]	Loss: 0.152626
Train Epoch: 0 [12800/26500 (48%)]	Loss: 0.092195
Train Epoch: 0 [16000/26500 (60%)]	Loss: 0.007857
Train Epoch: 0 [19200/26500 (72%)]	Loss: 0.016647
Train Epoch: 0 [22400/26500 (84%)]	Loss: 0.727773
Train Epoch: 0 [25600/26500 (97%)]	Loss: 0.650863
Train Epoch: 1 [0/26500 (0%)]	Loss: 0.022962
Train Epoch: 1 [3200/26500 (12%)]	Loss: 0.012848
Train Epoch: 1 [6400/26500 (24%)]	Loss: 0.194198
Train Epoch: 1 [9600/26500 (36%)]	Loss: 0.393200
Train Epoch: 1 [12800/26500 (48%)]	Loss: 0.888061
Train Epoch: 1 [16000/26500 (60%)]	Loss: 0.033129
Train Epoch: 1 [19200/26500 (72%)]	Loss: 0.183562
Train Epoch: 1 [22400/26500 (84%)]	Loss: 0.011540
Train Epoch: 1 [25600/26500 (97%)]	Loss: 0.003465
Train Epoch: 2 [0/26500 (0%)]	Loss: 0.023933
Train Epoch: 2 [3200/26500 (12%)]	Loss: 0.018774
Train Epoch: 

Train Epoch: 8 [6400/26501 (24%)]	Loss: 0.006368
Train Epoch: 8 [9600/26501 (36%)]	Loss: 0.380614
Train Epoch: 8 [12800/26501 (48%)]	Loss: 0.231013
Train Epoch: 8 [16000/26501 (60%)]	Loss: 0.535803
Train Epoch: 8 [19200/26501 (72%)]	Loss: 0.029590
Train Epoch: 8 [22400/26501 (84%)]	Loss: 0.042992
Train Epoch: 8 [25600/26501 (97%)]	Loss: 0.022863
Train Epoch: 9 [0/26501 (0%)]	Loss: 0.015605
Train Epoch: 9 [3200/26501 (12%)]	Loss: 0.006647
Train Epoch: 9 [6400/26501 (24%)]	Loss: 0.018759
Train Epoch: 9 [9600/26501 (36%)]	Loss: 0.006922
Train Epoch: 9 [12800/26501 (48%)]	Loss: 0.013079
Train Epoch: 9 [16000/26501 (60%)]	Loss: 0.063167
Train Epoch: 9 [19200/26501 (72%)]	Loss: 0.096608
Train Epoch: 9 [22400/26501 (84%)]	Loss: 0.008845
Train Epoch: 9 [25600/26501 (97%)]	Loss: 0.219988

Test set: Average loss: 0.0987, Accuracy: 6477/6625 (98%), AUROC: 0.7766

Test loss:  0.09872204844119414
AUROC:  0.7765666636646341
Fold:  3
Train Epoch: 0 [0/26501 (0%)]	Loss: 2.716161
Train Epoch: 0 [3200/2

### Make predictions for the test data

In [ ]:
def predict(network, test_loader):
    network.eval()
    
    probs = np.zeros((len(test_loader.dataset)))
    preds = np.zeros((len(test_loader.dataset)))
    for i, data in enumerate(test_loader):  # return each batch
        data = data.to(device)
        
        output = network(data)
    
        prob = F.softmax(output, 1)[:, 1].cpu().detach().numpy()
        pred = output.data.max(1, keepdim=True)[1].flatten().cpu().detach().numpy() # get the index of the max log-probability
        
        probs[i*batch_size:(i+1)*batch_size] = prob
        preds[i*batch_size:(i+1)*batch_size] = pred
        
    return probs, preds    

In [ ]:
# load all of the trained models
model1 = timm.create_model('tf_efficientnet_b2_ns', pretrained=True, num_classes=2)
model1.load_state_dict(torch.load("./enet1.pt"))
model1.cuda()

model2 = timm.create_model('tf_efficientnet_b2_ns', pretrained=True, num_classes=2)
model2.load_state_dict(torch.load("./enet2.pt"))
model2.cuda()

model3 = timm.create_model('tf_efficientnet_b2_ns', pretrained=True, num_classes=2)
model3.load_state_dict(torch.load("./enet3.pt"))
model3.cuda()

model4 = timm.create_model('tf_efficientnet_b2_ns', pretrained=True, num_classes=2)
model4.load_state_dict(torch.load("./enet4.pt"))
model4.cuda()


model5 = timm.create_model('tf_efficientnet_b2_ns', pretrained=True, num_classes=2)
model5.load_state_dict(torch.load("./enet5.pt"))
model5.cuda()


testset = MyDataset(test_df, train=False, transform=test_transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=4)
p1, _ = predict(model1, test_loader)
p2, _ = predict(model2, test_loader)
p3, _ = predict(model3, test_loader)
p4, _ = predict(model4, test_loader)
p5, _ = predict(model5, test_loader)
probs = (p1 + p2 + p3 + p4 + p5) / 5

In [ ]:
# ensemble the models into a single model
class MyEnsemble(nn.Module):
    def __init__(self, model1, model2, model3, model4, model5):
        super(MyEnsemble, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4
        self.model5 = model5
        self.classifier = nn.Linear(10, 2)
        
    def forward(self, img):
        x1 = self.model1(img)
        x2 = self.model2(img)
        x3 = self.model3(img)
        x4 = self.model4(img)
        x5 = self.model5(img)
        
        x = torch.cat((x1, x2, x3, x4, x5), dim=1)
        x = self.classifier(F.relu(x))
        return x
        
model = MyEnsemble(model1, model2, model3, model4, model5)   

In [ ]:
# predict for the test_values
# probs, preds = predict(network, test_loader)

In [ ]:
# write to file
submission = pd.DataFrame({'image_name': test_df['image_name'], 'target': probs})

submission.to_csv('enet.csv', index=False)

### Get evaluation metrics

In [ ]:
%%time

# make evaluations on all the training data
test_loss, test_acc, test_roc = test(network, train_loader)  # print the validation error

### Store Trained Model

In [ ]:
# save the model
PATH = "enet.pt"
torch.save(network.state_dict(), PATH)  # we only save the state_dict rather than entire model

In [ ]:
# load the model and use for evaluation
network = MultiLayerNet(num_inputs, num_outputs, hidden_units=1000)
#model.load_state_dict("../output/" + torch.load(PATH)) 
network.load_state_dict(torch.load("../input/multilayer-model/multilayer.pt")) 
network.eval()  # set dropout and batch normalization layers to evaluation before running inference

### Predicting for a Single Image

In [ ]:
def predict_single_image(image):
    """Given an image, predict benign or malignant """
    network.eval()
    
    for i, data in enumerate(test_loader):  # return each batch
        output = network(data)
    
        prob = F.softmax(output)[:, 1].item()
        pred = output.data.max(1, keepdim=True)[1].flatten().item() # get the index of the max log-probability
        
    return prob, pred